In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PositionalEmbedding3D import PositionalEmbedding3D

In [8]:
import torch
import torch.nn as nn

class MinecraftSequencePredict(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, src_shape: tuple, tgt_shape: tuple, tgt_offset: tuple):
        super(MinecraftSequencePredict, self).__init__()

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model)

        self.pos_embedding = PositionalEmbedding3D(d_model, src_shape, tgt_shape, tgt_offset)
        # Define the transformer model
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            batch_first=True
        )

        # Define the output layer
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, src, tgt):
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        
        src = self.pos_embedding(src, True)
        tgt = self.pos_embedding(tgt, False)
        output = self.transformer(src, tgt)

        # You may want to reshape or slice the output based on your specific task
        output = self.fc(output)

        return output

# Example usage
vocab_size = 255  # Adjust based on your vocabulary size
d_model = 768
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

src_shape  = (5,5,5)
tgt_shape  = (1,5,5)
tgt_offset = (5,0,0)


# Set the seed for reproducibility (optional)
torch.manual_seed(42)

# Create a 5x5x5 tensor with random integers from 0 to 255
src = torch.randint(0, vocab_size - 1, size=src_shape, dtype=torch.int).ravel().unsqueeze(0)
tgt = torch.cat([torch.tensor([vocab_size-1]),torch.randint(0, vocab_size - 1, size=tgt_shape, dtype=torch.int).ravel()]).unsqueeze(0)
tgt_sos = torch.tensor([vocab_size-1]).unsqueeze(0)


model = MinecraftSequencePredict(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, src_shape, tgt_shape, tgt_offset)

output = model(src, tgt)
print(output.shape)
torch.argmax(output)

[(5, 0, 0) (5, 0, 1) (5, 0, 2) (5, 0, 3) (5, 0, 4) (5, 1, 0) (5, 1, 1)
 (5, 1, 2) (5, 1, 3) (5, 1, 4) (5, 2, 0) (5, 2, 1) (5, 2, 2) (5, 2, 3)
 (5, 2, 4) (5, 3, 0) (5, 3, 1) (5, 3, 2) (5, 3, 3) (5, 3, 4) (5, 4, 0)
 (5, 4, 1) (5, 4, 2) (5, 4, 3) (5, 4, 4)]
<class 'list'> <class 'list'> <class 'list'> 
tensor([6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5])
tensor([5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4,
        4, 4])
tensor([5, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4]) 
torch.Size([125, 256])
torch.Size([125, 256])
torch.Size([125, 256])

torch.Size([125, 768])
torch.Size([26, 768])
torch.Size([1, 26, 255])


tensor(3879)

In [10]:
# Example usage
vocab_size = 255  # Adjust based on your vocabulary size
d_model = 768
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6

src_shape  = (5,5,5)
tgt_shape  = (1,5,5)
tgt_offset = (5,0,0)


# Set the seed for reproducibility (optional)
torch.manual_seed(42)

# Create a 5x5x5 tensor with random integers from 0 to 255
src = torch.randint(0, vocab_size - 1, size=src_shape, dtype=torch.int).ravel().unsqueeze(0)
tgt = torch.cat([torch.tensor([vocab_size-1]),torch.randint(0, vocab_size - 1, size=tgt_shape, dtype=torch.int).ravel()]).unsqueeze(0)

print(src)
print(tgt)

tensor([[170,  67, 148, 136, 200, 159, 242, 104,  24, 201,  84, 236,  94, 110,
         185, 182,  31, 175, 217, 132, 223, 102,   9,  79, 247,  61,   1,  97,
         143, 231, 142, 112, 109,  23, 137, 120, 155, 250, 188, 182, 122,  64,
          93, 183,  89, 201, 177,  24, 111,  55, 156,  11, 232,  51, 170, 189,
         157,  62, 202, 130,  86, 232,   2,  16,  58, 107,  37,   6,  37, 181,
         131,  15, 190, 141,  22, 131,  73, 203,  92, 157, 132, 179, 220, 117,
         138,  78, 183,   2, 133,  81,  73, 209,  77, 205, 157,  37,  15, 135,
          59, 120, 206, 205,  11, 131,  65, 187,  17,  73,  17, 242, 131, 106,
         253, 195, 246,  77, 172, 119, 155,  46, 167, 240, 159, 242, 202]],
       dtype=torch.int32)
tensor([[254,  77,  41, 252, 145, 107, 213, 136, 208, 138, 136, 154, 176, 248,
          12,  22,   3, 242,  17, 129,  85,  62,  78, 212,  10, 175]])


In [4]:
import numpy as np

# position = torch.arange(0, x.size(1)).unsqueeze(0)
shape = (4,4,4)

array_3d = np.empty(shape, dtype=object)

# Fill in the array with tuples of indices
for i in range(shape[0]):
    for j in range(shape[1]):
        for k in range(shape[2]):
            array_3d[i, j, k] = (i, j, k)
array_3d = array_3d.ravel()
print(array_3d)
arr = [i[0] for _, i in np.ndenumerate(array_3d)]
arr.insert(0, 12)
print(type(arr))

[(0, 0, 0) (0, 0, 1) (0, 0, 2) (0, 0, 3) (0, 1, 0) (0, 1, 1) (0, 1, 2)
 (0, 1, 3) (0, 2, 0) (0, 2, 1) (0, 2, 2) (0, 2, 3) (0, 3, 0) (0, 3, 1)
 (0, 3, 2) (0, 3, 3) (1, 0, 0) (1, 0, 1) (1, 0, 2) (1, 0, 3) (1, 1, 0)
 (1, 1, 1) (1, 1, 2) (1, 1, 3) (1, 2, 0) (1, 2, 1) (1, 2, 2) (1, 2, 3)
 (1, 3, 0) (1, 3, 1) (1, 3, 2) (1, 3, 3) (2, 0, 0) (2, 0, 1) (2, 0, 2)
 (2, 0, 3) (2, 1, 0) (2, 1, 1) (2, 1, 2) (2, 1, 3) (2, 2, 0) (2, 2, 1)
 (2, 2, 2) (2, 2, 3) (2, 3, 0) (2, 3, 1) (2, 3, 2) (2, 3, 3) (3, 0, 0)
 (3, 0, 1) (3, 0, 2) (3, 0, 3) (3, 1, 0) (3, 1, 1) (3, 1, 2) (3, 1, 3)
 (3, 2, 0) (3, 2, 1) (3, 2, 2) (3, 2, 3) (3, 3, 0) (3, 3, 1) (3, 3, 2)
 (3, 3, 3)]
<class 'list'>
